In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



iris = load_iris()
x=iris['data']
y=iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, stratify=y)

In [2]:
scaler = StandardScaler()
scaler.fit(x_train) #평균, 표준편차 이런값들을 변수에 저장
x_train_tensor = torch.from_numpy(scaler.transform(x_train)).float()
x_test_tensor = torch.from_numpy(scaler.transform(x_test)).float()

y_train_tensor = torch.from_numpy(y_train).long()
y_test_tensor = torch.from_numpy(y_test).long()

print (x_train_tensor.shape, x_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape)

torch.Size([120, 4]) torch.Size([30, 4]) torch.Size([120]) torch.Size([30])


In [6]:
batch_size = 120

In [7]:
class mymodel(nn.Module):
    def __init__(self,i,o):
        super().__init__()

        self.go = nn.Sequential(
             nn.Linear(i, 100),
            nn.LeakyReLU(0.1),
            nn.Linear(100, 20),
            nn.LeakyReLU(0.1),
            nn.Linear(20, 5),
            nn.LeakyReLU(0.1),
            nn.Linear(5, o),
            # nn.Softmax(dim=-1)
            nn.LogSoftmax(dim=-1)
        )

    def forward(self,x):
        return self.go(x)

In [8]:
input_dim = x_train_tensor.size(-1)
output_dim = 3
print(input_dim, output_dim)

4 3


In [9]:
model = mymodel(input_dim,output_dim)
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

In [10]:
for i in range(1000):
    indices = torch.randperm(x_train_tensor.size(0))

    x_batch_list = torch.index_select(x_train_tensor,0,indices).split(batch_size,0)
    y_batch_list = torch.index_select(y_train_tensor,0,indices).split(batch_size,0)
    temp = []
    for x,y in zip(x_batch_list, y_batch_list): #xbatch list tensor[256][~] 가 여러개 존재
        y_pred = model(x)
        loss = loss_func(y_pred,y)
        temp.append(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(i%100)==0:
        print(sum(temp)/len(temp))

tensor(1.0696, grad_fn=<DivBackward0>)
tensor(0.2391, grad_fn=<DivBackward0>)
tensor(0.0582, grad_fn=<DivBackward0>)
tensor(0.0366, grad_fn=<DivBackward0>)
tensor(0.0269, grad_fn=<DivBackward0>)
tensor(0.0174, grad_fn=<DivBackward0>)
tensor(0.0093, grad_fn=<DivBackward0>)
tensor(0.0047, grad_fn=<DivBackward0>)
tensor(0.0026, grad_fn=<DivBackward0>)
tensor(0.0016, grad_fn=<DivBackward0>)


In [11]:
model.eval()
with torch.no_grad():
    y_test_pred = model(x_test_tensor)
    y_pred_list = torch.argmax(y_test_pred, dim=1)

In [12]:
y_pred_list = list()
x_test_batch_list = x_test_tensor.split(batch_size, 0)
model.eval()
with torch.no_grad():
    for x_minibatch in x_test_batch_list:
        y_test_pred = model(x_minibatch)
        y_test_pred = torch.argmax(y_test_pred, dim=1)
        y_pred_list.extend(y_test_pred.detach().tolist())

y_pred_list = torch.tensor(y_pred_list)

In [13]:
print (y_pred_list.shape, y_test_tensor.shape)

torch.Size([30]) torch.Size([30])


In [15]:
y_test_pred

tensor([1, 2, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2,
        0, 2, 0, 1, 2, 2])

In [16]:
x_train_tensor

tensor([[ 0.3749, -0.5081,  0.1552,  0.1354],
        [-1.0843,  0.1795, -1.2862, -1.4498],
        [ 0.7397, -0.5081,  1.0777,  1.3243],
        [ 0.7397, -0.5081,  1.0777,  1.1922],
        [-0.2331, -1.1956,  0.0975, -0.1288],
        [ 1.1046,  0.1795,  0.3858,  0.2675],
        [ 1.1046,  0.6379,  1.1354,  1.1922],
        [-1.6923,  0.4087, -1.4015, -1.3177],
        [ 0.0101, -0.7372,  0.7894,  0.9280],
        [-0.3547, -1.1956,  0.1552,  0.1354],
        [ 0.1317, -0.0497,  0.7894,  0.7959],
        [-0.8411,  1.0963, -1.3439, -1.1856],
        [-1.5707, -1.6540, -1.4015, -1.1856],
        [-0.8411, -1.1956, -0.4214, -0.1288],
        [-0.9627,  0.8671, -1.2862, -1.3177],
        [ 1.1046,  0.1795,  0.5588,  0.3996],
        [-0.9627,  0.8671, -1.2286, -1.0535],
        [ 1.2262, -0.5081,  0.6164,  0.2675],
        [ 0.9830, -0.2789,  0.5011,  0.1354],
        [-1.0843,  0.1795, -1.2862, -1.3177],
        [-0.9627,  0.6379, -1.3439, -1.3177],
        [-1.2059,  0.8671, -1.2286